# Simulate a feedback survey
This notebook provides sample EDSL code for simulating surveys with AI agents and large language models.
In the steps below we use EDSL to prompt LLMs to suggest names for a yoga studio, and then simulate a feedback survey with AI agents representing target customers.

Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

## Create a question
We start by creating a question to prompt a language model to suggest some names for a yoga studio.
EDSL comes with many common [question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from the model - multiple choice, free text, linear scale, etc. 
Here we use `QuestionList` to prompt a model to return a list of items:

In [1]:
from edsl import QuestionList

In [2]:
q = QuestionList(
    question_name = "yoga_studio_name",
    question_text = "What are some creative names for a yoga studio?",
    max_list_items = 10
)

## Select some models to answer the question
EDSL works with many popular [language models that we can select](https://docs.expectedparrot.com/en/latest/language_models.html) to generate responses to questions.

In [3]:
from edsl import ModelList, Model

To see a list of all services:

In [4]:
Model.services()

,Service Name
0,anthropic
1,azure
2,bedrock
3,deep_infra
4,deepseek
5,google
6,groq
7,mistral
8,ollama
9,openai


A list of current available models can be viewed [here](https://www.expectedparrot.com/getting-started/coop-pricing).

To select models to use with a question or survey:

In [5]:
models = ModelList(
    Model(m) for m in ["claude-3-sonnet-20240229", "gpt-4o"]
)

## Run the question 
We administer a question to a language model by calling the `run()` method on it.
This generates a formatted dataset of `Results`:

In [6]:
results = q.by(models).run()

Job UUID,f7aba60b-c169-4373-b934-99c4eae28eb3
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/f7aba60b-c169-4373-b934-99c4eae28eb3
Exceptions Report URL,None
Results UUID,6a1de3ab-134c-4f93-9812-656b3055a48d
Results URL,https://www.expectedparrot.com/content/6a1de3ab-134c-4f93-9812-656b3055a48d


## Inspect the results
EDSL comes with built-in [methods for analyzing results](https://docs.expectedparrot.com/en/latest/results.html).
Here we inspect the responses for each model:

In [7]:
results.select("model", "yoga_studio_name")

,model.model,answer.yoga_studio_name
0,claude-3-sonnet-20240229,"['Zen Oasis', 'Mindful Movements', 'Serenity Studio', 'Breathe Bliss', 'Harmony Haven', 'Peaceful Postures', 'Tranquil Tribe', 'Soul Stretch', 'Chakra Chimes', 'Yogic Vibes']"
1,gpt-4o,"['Zen Flow', 'Soul Stretch', 'Namaste Nest', 'Peaceful Poses', 'Harmony Haven', 'Serenity Studio', 'Mindful Movements', 'Tranquil Transitions', 'Blissful Balance', 'Inner Light Yoga']"


## Use the responses in new questions
We can format the responses to use them as options to new questions:

In [8]:
claude_names = results.filter("model.model == 'claude-3-sonnet-20240229'").select("yoga_studio_name").to_list()[0]
claude_names

['Zen Oasis',
 'Mindful Movements',
 'Serenity Studio',
 'Breathe Bliss',
 'Harmony Haven',
 'Peaceful Postures',
 'Tranquil Tribe',
 'Soul Stretch',
 'Chakra Chimes',
 'Yogic Vibes']

In [9]:
gpt4o_names = results.filter("model.model == 'gpt-4o'").select("yoga_studio_name").to_list()[0]
gpt4o_names

['Zen Flow',
 'Soul Stretch',
 'Namaste Nest',
 'Peaceful Poses',
 'Harmony Haven',
 'Serenity Studio',
 'Mindful Movements',
 'Tranquil Transitions',
 'Blissful Balance',
 'Inner Light Yoga']

We can optionally randomize the list of options:

In [10]:
studio_names = list(set(gpt4o_names + claude_names))

In [11]:
import random

random.shuffle(studio_names)

## Constructing a survey
Here we create some new questions in different types and combine them into a survey to administer them together:

In [12]:
from edsl import QuestionMultipleChoice, QuestionCheckBox, Survey

In [13]:
q1 = QuestionMultipleChoice(
    question_name = "favorite",
    question_text = "What is your favorite name for a yoga studio?",
    question_options = studio_names
)

In [14]:
q2 = QuestionCheckBox(
    question_name = "pick",
    question_text = "Pick the 3 best names for a yoga studio.",
    question_options = studio_names,
    min_selections = 3,
    max_selections = 3
)

In [15]:
survey = Survey([q1, q2])

## Designing agents to answer the survey
Next we can design AI agents with relevant traits to answer the questions.
Here we use a model to draft some personas, and then create "agent" objects for them to use with the survey:

In [16]:
q = QuestionList(
    question_name = "personas",
    question_text = "Draft 5 diverse personas for patrons of a yoga studio."
)

In [17]:
personas = q.run().select("personas").to_list()[0]
personas

Job UUID,ede4170d-8e73-469d-85b5-021e9b909fe3
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/ede4170d-8e73-469d-85b5-021e9b909fe3
Exceptions Report URL,None
Results UUID,80a43b5c-e02d-4302-98c2-d41e14a6e2a1
Results URL,https://www.expectedparrot.com/content/80a43b5c-e02d-4302-98c2-d41e14a6e2a1


['A busy corporate executive seeking stress relief and work-life balance',
 'A retired senior looking to maintain flexibility and social connections',
 'A college student exploring mindfulness and physical fitness',
 'A new parent using yoga to regain strength and mental clarity',
 'An athlete incorporating yoga for cross-training and injury prevention']

In [18]:
from edsl import AgentList, Agent

In [19]:
agents = AgentList(
    Agent(traits = {"persona":p}) for p in personas
)

### Run the survey with the agents
Here we add the agents to the survey and run it with the language models we selected above (to compare responses):

In [20]:
results = survey.by(agents).by(models).run()

Job UUID,869d24a1-a8d2-4cfc-b2d7-916096fd87d7
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/869d24a1-a8d2-4cfc-b2d7-916096fd87d7
Exceptions Report URL,None
Results UUID,a9b19bd4-2370-4617-b1f7-bc4a38fa2da2
Results URL,https://www.expectedparrot.com/content/a9b19bd4-2370-4617-b1f7-bc4a38fa2da2


In [21]:
(
    results
    .sort_by("model", "persona")
    .select("model", "persona", "favorite", "pick")
)

,model.model,agent.persona,answer.favorite,answer.pick
0,claude-3-sonnet-20240229,A busy corporate executive seeking stress relief and work-life balance,Zen Oasis,"['Blissful Balance', 'Breathe Bliss', 'Serenity Studio']"
1,claude-3-sonnet-20240229,A college student exploring mindfulness and physical fitness,Tranquil Tribe,"['Blissful Balance', 'Zen Flow', 'Tranquil Tribe']"
2,claude-3-sonnet-20240229,A new parent using yoga to regain strength and mental clarity,Tranquil Transitions,"['Blissful Balance', 'Harmony Haven', 'Serenity Studio']"
3,claude-3-sonnet-20240229,A retired senior looking to maintain flexibility and social connections,Tranquil Transitions,"['Tranquil Transitions', 'Peaceful Postures', 'Serenity Studio']"
4,claude-3-sonnet-20240229,An athlete incorporating yoga for cross-training and injury prevention,Tranquil Transitions,"['Blissful Balance', 'Zen Flow', 'Tranquil Tribe']"
5,gpt-4o,A busy corporate executive seeking stress relief and work-life balance,Blissful Balance,"['Blissful Balance', 'Harmony Haven', 'Serenity Studio']"
6,gpt-4o,A college student exploring mindfulness and physical fitness,Zen Flow,"['Blissful Balance', 'Inner Light Yoga', 'Serenity Studio']"
7,gpt-4o,A new parent using yoga to regain strength and mental clarity,Namaste Nest,"['Tranquil Transitions', 'Blissful Balance', 'Harmony Haven']"
8,gpt-4o,A retired senior looking to maintain flexibility and social connections,Tranquil Transitions,"['Tranquil Transitions', 'Harmony Haven', 'Serenity Studio']"
9,gpt-4o,An athlete incorporating yoga for cross-training and injury prevention,Zen Flow,"['Soul Stretch', 'Blissful Balance', 'Zen Flow']"


We could also solicit feedback on individual names.
This can be done by creating a "scenario" of each question for each name:

In [22]:
from edsl import QuestionLinearScale, QuestionFreeText, Survey, ScenarioList

In [23]:
scenarios = ScenarioList.from_list("studio_name", studio_names)
scenarios.table()

,studio_name
0,Tranquil Transitions
1,Soul Stretch
2,Yogic Vibes
3,Peaceful Postures
4,Blissful Balance
5,Namaste Nest
6,Harmony Haven
7,Breathe Bliss
8,Inner Light Yoga
9,Zen Flow


In [24]:
q_memorable = QuestionLinearScale(
    question_name = "memorable",
    question_text = "How memorable is this yoga studio name: {{ scenario.studio_name }}",
    question_options = [1,2,3,4,5],
    option_labels = {1:"Not at all memorable", 5:"Very memorable"}
)

In [25]:
q_criticism = QuestionFreeText(
    question_name = "criticism",
    question_text = "What are some criticisms people might have about this yoga studio name: {{ scenario.studio_name }}"
)

In [26]:
survey = Survey([q_memorable, q_criticism])

In [27]:
results = survey.by(scenarios).by(agents).by(models).run()

Job UUID,2253cbdf-bac9-441a-b1c5-e1d44e6641de
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/2253cbdf-bac9-441a-b1c5-e1d44e6641de
Exceptions Report URL,None
Results UUID,29cda4f1-fe65-42d5-a8e1-5cef87920836
Results URL,https://www.expectedparrot.com/content/29cda4f1-fe65-42d5-a8e1-5cef87920836


In [28]:
(
    results
    .sort_by("model", "persona")
    .select("model", "persona", "studio_name", "memorable", "criticism")
)

,model.model,agent.persona,scenario.studio_name,answer.memorable,answer.criticism
0,claude-3-sonnet-20240229,A busy corporate executive seeking stress relief and work-life balance,Tranquil Transitions,4,"As a busy corporate executive seeking stress relief and work-life balance, here are some potential criticisms I could see about the yoga studio name ""Tranquil Transitions"": • It's a bit generic and vague. There are a lot of yoga studios with names involving words like ""tranquil"" or ""transitions."" It doesn't really stand out or give a strong sense of the studio's unique identity. • The word ""transitions"" could be confusing or offputting to some. It makes me wonder if the studio specializes in some sort of transitional style of yoga rather than being a more general studio. • While ""tranquil"" conveys a sense of peace and relaxation, it may come across as boring or unmemorable to some potential customers looking for something more dynamic or inspiring. • The name doesn't give any indication of what style or types of yoga the studio offers. As someone very new to yoga, I'd want the name to give me a better sense of what to expect. Overall, while not a terrible name, it doesn't particularly excite me or make me really want to rush out and try this specific studio over others in the area. A more distinctive and evocative name highlighting the studio's unique approach or community could be more appealing to a time-crunched executive like myself."
1,claude-3-sonnet-20240229,A busy corporate executive seeking stress relief and work-life balance,Soul Stretch,4,"As a busy corporate executive seeking stress relief and work-life balance, here are some potential criticisms I could see with the yoga studio name ""Soul Stretch"": • It sounds a bit new age-y or woo-woo, which could turn off more pragmatic or skeptical clients looking for just the physical exercise benefits of yoga. The ""soul"" reference may come across as too spiritual for some. • The name is a little vague and doesn't give a strong sense of what type of yoga or experience to expect. It doesn't differentiate the studio from others. • ""Soul Stretch"" could be seen as trying too hard to be catchy or clever with the alliteration. Sometimes simpler and more straightforward names can be more appealing. • For a corporate professional, the name may not inspire confidence that this is a serious, high-quality studio focused on challenging physical practice. It could seem more suited to a hippie studio vibe. However, the name could also resonate well if positioned properly as a holistic mind-body experience. But those are some potential criticisms I could envision from a busy executive's perspective just looking for an effective workout amongst all my corporate responsibilities. A more neutral name may come across as more credible."
2,claude-3-sonnet-20240229,A busy corporate executive seeking stress relief and work-life balance,Yogic Vibes,4,"As a busy corporate executive seeking stress relief and work-life balance, here are some potential criticisms I might have about the yoga studio name ""Yogic Vibes"": 1. Overused terminology: The words ""yogic"" and ""vibes"" are quite trendy and commonly used in the yoga/wellness space. The name may come across as unoriginal or clichéd. 2. Lack of distinctiveness: With so many yoga studios using similar new-age, spiritual terminology in their names, ""Yogic Vibes"" may not stand out or differentiate itself enough in the market. 3. Vague meaning: While the name hints at a relaxing, peaceful atmosphere, it doesn't provide much specific information about the studio's offerings, teaching style, or unique value proposition. 4. Unprofessional tone: As an executive, I might perceive ""Yogic Vibes"" as having a too casual, hippy-ish vibe that doesn't align with the more serious, professional environment I'm used to in the corporate world. 5. Limited appeal: The name could potentially turn off certain demographics like older adults or men who m

## Posting to the Coop
The [Coop](https://www.expectedparrot.com/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we demonstrate how to post this notebook to share with others (visibility can be *public*, *private* or *unlisted* by default):

In [30]:
from edsl import Notebook

nb = Notebook(path = "yoga_studio_name_survey.ipynb")

if refresh := False:
    nb.push(
        description = "Feedback survey", 
        alias = "feedback-survey-notebook",
        visibility = "public"
    )
else:
    nb.patch("https://www.expectedparrot.com/content/RobinHorton/feedback-survey-notebook", value = nb)